In [1]:
#Ryan Kittelson CS-340 Project Two Dashboard

from jupyter_plotly_dash import JupyterDash
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser1"
password = "aacuser1"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'grazioso.png' # replaced with my own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Ryan Kittelson'))), #my name
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Hr(),
    html.Div(className='row',
            style={'display':'flex'},
            children=[
                html.Button(id='submit-button-one', n_clicks=0, children='Water Rescue'),
                html.Button(id='submit-button-two', n_clicks=0, children='Mountain or Wilderness Rescue'),
                html.Button(id='submit-button-three', n_clicks=0, children='Disaster or Individual Tracking'),
                html.Button(id='submit-button-four', n_clicks=0, children='Reset')
            ]),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable = False,
        selected_columns =[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,

    ),
     html.Br(),
     html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id', "data"),
             [Input('submit-button-one', 'n_clicks'), Input('submit-button-two', 'n_clicks'),
              Input('submit-button-three', 'n_clicks'), Input('submit-button-four', 'n_clicks'),
             ])

def on_click(bt1,bt2,bt3,bt4):
    #start case
    if (int(bt1) == 0 and int(bt2) == 0 and int(bt3) == 0 and int(bt4) == 0):
        df = pd.DataFrame.from_records(shelter.read_all({}))
    #use higher number of button clicks to determine filter type
    elif (int(bt1) > int(bt2)): # Water Rescue
        df = pd.DataFrame(list(shelter.read_all({'$and' : [{'sex_upon_outcome': 'Intact Female'},
                                                          {'$or': [
                                                              {'breed': 'Chesa Bay Retr Mix'},
                                                              {'breed': 'Labrador Retriever Mix'},
                                                              {'breed': 'Newfoundland/Labrador Retriever'},
                                                              {'breed': 'Newfoundland Mix'},
                                                              {'breed': 'Newfoundland/Australian Cattle Dog'},
                                                              {'breed': 'Newfoundland/Great Pyrenees'}]
                                                          },
                                                           {'$and': [{'age_upon_outcome_in_weeks': {'$gte':26}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                           }]
                                                          })))
    elif (int(bt2) > int(bt1)): # Mountain or Wilderness Rescue
        df = pd.DataFrame(list(shelter.read_all({'$and' : [{'sex_upon_outcome': 'Intact Male'},
                                                          {'$or': [
                                                              {'breed': 'Alaskan Malamute'},
                                                              {'breed': 'Old English Sheepdog'},
                                                              {'breed': 'German Shepherd'},
                                                              {'breed': 'Siberian Husky'},
                                                              {'breed': 'Rottweiler'}]
                                                          },
                                                           {'$and': [{'age_upon_outcome_in_weeks': {'$gte':26}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                           }]
                                                          })))
    elif (int(bt3) > int(bt1)): # Disaster or Individual Tracking Rescue
        df = pd.DataFrame(list(shelter.read_all({'$and' : [{'sex_upon_outcome': 'Intact Male'},
                                                          {'$or': [
                                                              {'breed': 'Doberman Pinsch'},
                                                              {'breed': 'German Shepherd'},
                                                              {'breed': 'Golden Retriever'},
                                                              {'breed': 'Rottweiler'},
                                                              {'breed': 'Bloodhound'}]
                                                          },
                                                           {'$and': [{'age_upon_outcome_in_weeks': {'$gte':20}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                           }]
                                                          })))
    elif (int(bt4) > int(bt1)):
        df = pd.DataFrame(list(shelter.read_all({"animal_type":"Dog"})))
    return df.to_dict('records')



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

#def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    viewDF = pd.DataFrame.from_dict(viewData)
    dff = viewDF.loc[rows]
    # Austin TX is at [30.75,-97.48]
    return [ dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=15, children=[
        dl.TileLayer(id="base-layer-id"),
# Marker with tool tip and pop up
        dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
            dl.Tooltip(dff['breed']),
            dl.Popup([
                 html.H1("Animal Name"),
                 html.P(dff.loc[0,'name'])
            ])
        ])
])]


app

NameError: name 'dash_table' is not defined